In [2]:
from fastai.vision.all import *
import re
import pickle
import numpy as np
import PIL

# Function to unpickle the dataset
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict 

/opt/conda/envs/fastai/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729096996/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
path = Path()/'CIFAR10/'

In [4]:
filenames = path.ls()
print(filenames)
train_path = filenames[1:6]
valid_path = filenames[-1]

[Path('CIFAR10/batches.meta'), Path('CIFAR10/data_batch_1'), Path('CIFAR10/data_batch_2'), Path('CIFAR10/data_batch_3'), Path('CIFAR10/data_batch_4'), Path('CIFAR10/data_batch_5'), Path('CIFAR10/readme.html'), Path('CIFAR10/test_batch')]


In [5]:
# Upack all the training batches from the original dataset 
train_batch = [unpickle(items) for items in train_path]
# Upack all the images stored as numpy array from the training dataset.
train_x = [batches[b'data'] for batches in train_batch]
train_y = [batches[b'labels'] for batches in train_batch]

In [7]:
# Reshaping and converting images to tensor
def reshape(im_array):
    im_re = [np.transpose(np.reshape(items,[3, 32, 32])) for items in im_array ]
    return im_re

im_list = [reshape(x) for x in train_x]
print(im_list)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [178]:
tensor_x = [tensor(Image.fromarray(images, 'RGB')) for images in im_list]

In [180]:
tensor_x[1]

5

In [120]:
im_array = batch_array[4]
print(im_array)
im_re = np.transpose(np.reshape(im_array,[3, 32, 32]))
print(im_re)
im = Image.fromarray(im_re, 'RGB')

/bin/bash: -c: line 0: syntax error near unexpected token `'something.png''
/bin/bash: -c: line 0: `rm ('something.png')'
